# Capitaux 03: Full Pipeline Testing

**Purpose**: Test complete Capitaux pipeline (AZ + AZEC → Gold)

**Tests**:
1. Run AZCapitauxProcessor
2. Run AZECCapitauxProcessor  
3. Run CapitauxConsolidationProcessor
4. Verify output dataset

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("Capitaux_Pipeline_Testing") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Initialize Processors

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from utils.logger import PipelineLogger

config = ConfigLoader(str(project_root / "config" / "config.yml"))
logger = PipelineLogger("capitaux_test")

VISION = "202509"
print(f"Testing pipeline for vision: {VISION}")

## 2. Run AZ Capitaux Processor

In [ ]:
from src.processors.capitaux_processors import AZCapitauxProcessor

try:
    az_processor = AZCapitauxProcessor(spark, config, logger)
    df_az = az_processor.run(VISION)
    
    if df_az is not None:
        print(f"✓ AZ Capitaux: {df_az.count():,} rows")
        df_az.select('nopol', 'smp_100', 'lci_100').show(5)
    else:
        print("⚠ AZ Processor returned None")
except Exception as e:
    print(f"✗ AZ Processor error: {e}")
    df_az = None

## 3. Run AZEC Capitaux Processor

In [ ]:
from src.processors.capitaux_processors import AZECCapitauxProcessor

try:
    azec_processor = AZECCapitauxProcessor(spark, config, logger)
    df_azec = azec_processor.run(VISION)
    
    if df_azec is not None:
        print(f"✓ AZEC Capitaux: {df_azec.count():,} rows")
        df_azec.show(5)
    else:
        print("⚠ AZEC Processor returned None (data may not exist)")
except Exception as e:
    print(f"⚠ AZEC Processor: {e}")
    df_azec = None

## 4. Run Consolidation

In [ ]:
from src.processors.capitaux_processors import CapitauxConsolidationProcessor

try:
    consolid_processor = CapitauxConsolidationProcessor(spark, config, logger)
    df_gold = consolid_processor.run(VISION)
    
    if df_gold is not None:
        print(f"✓ Gold output: {df_gold.count():,} rows")
        print(f"  Columns: {len(df_gold.columns)}")
        df_gold.show(5)
    else:
        print("⚠ Consolidation returned None")
except Exception as e:
    print(f"✗ Consolidation error: {e}")
    df_gold = None

## Summary

In [ ]:
print("="*60)
print("CAPITAUX PIPELINE TESTING COMPLETE")
print("="*60)
print(f"\nVision: {VISION}")
print(f"AZ Capitaux:   {'✓' if df_az is not None else '✗'}")
print(f"AZEC Capitaux: {'✓' if df_azec is not None else '⚠ (optional)'}")
print(f"Gold output:   {'✓' if df_gold is not None else '✗'}")

if df_gold is not None:
    print(f"\nFinal gold dataset: {df_gold.count():,} rows")
    print("\n→ Run: python main.py --vision 202509 --component capitaux")